In [1]:
from pyspark.sql import SparkSession
import xgboost as xgb
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("xgb").getOrCreate()
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/27 01:32:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
events_shot = spark.read.csv('Datas/events_shot.csv', header=True, inferSchema=True)

In [4]:
features = ['other_pp','from_fk','from_ti','from_corner','from_counter','from_gk','from_keeper','from_ko',
            'header','corner_type','fk_type','pk_type',
            'half_volley_technique','volley_technique','lob_technique','overhead_technique','backheel_technique','diving_h_technique',
            'distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure',
            'shot_aerial_won','shot_first_time','shot_one_on_one','shot_open_goal','shot_follows_dribble','players_inside_area']
target = ['goal'] # sb_prediction, shot_statsbomb_xg, goal

In [5]:
df = events_shot.toPandas()

In [34]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error

target = 'goal'
X = df[features]
y = df[target]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=40)

# Create DMatrix for training data (only use training data)
dtrain = xgb.DMatrix(X_train, label=y_train)
# Create DMatrix for test data (only use test data)
dtest = xgb.DMatrix(X_test)

params = {
    'objective': 'binary:logistic',  # Binary classification objective
    'eval_metric': 'auc',            # Evaluation metric (AUC for binary classification)
    'learning_rate': 0.05,           # Slightly lower learning rate for finer updates
    'max_depth': 10,                 # Slightly deeper trees for capturing more complex patterns
    'subsample': 0.85,               # Slightly higher subsample for better data utilization
    'colsample_bytree': 0.9,         # Increase feature sampling for more diversity in trees
    'min_child_weight': 2,           # Slightly lower value to allow more splits if needed
    'gamma': 0.3,                    # Increase gamma to make the model more conservative
    'alpha': 0.2,                    # Increased L1 regularization to reduce overfitting
    'lambda': 1.5,                   # Increase L2 regularization to further prevent overfitting
    'scale_pos_weight': 1,           # Adjust this if you have class imbalance (optional)
    'nthread': 4,                    # Use multiple threads to speed up training (if applicable)
}

# Train the model
num_round = 500
model = xgb.train(params, dtrain, num_round)

# Make predictions
y_pred = model.predict(dtest)

# Calculate ROC-AUC score on test data
print(roc_auc_score(y_test, y_pred))

# Accuracy
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
print(accuracy_score(y_test, y_pred))

# rmse
rmse = mean_squared_error(y_test, y_pred)
print(rmse)

0.7932897669112459
0.9012741438078234
0.0987258561921766
